# R-model

Date of creation: 9.07.2020

Last updated: 10.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import NotebookLoader
import Physics as ph
import SSM18 as s

importing Jupyter notebook from Physics.ipynb
Physics version 1.12 11.07.2020
importing Jupyter notebook from SSM18.ipynb
SSM18 version 1.8 11.07.2020


In [3]:
def load_hydrogen():
    filename = 'Mass/hydrogen.csv'
    return pd.read_csv(filename)
epoch = 0
dfx = load_hydrogen()
dimM = len(dfx.index)
dfx

,Unnamed: 0,Epoch00,Epoch01
0,0,0.732,0.671493
1,1,0.732,0.674068
2,2,0.732,0.675536
3,3,0.732,0.676741
4,4,0.732,0.677796
...,...,...,...
996,996,0.732,0.732000
997,997,0.732,0.732000
998,998,0.732,0.732000
999,999,0.732,0.732000


In [4]:
epoch = 0
hyd = dfx['Epoch%02d' % epoch].values

In [23]:
"""
m is M/s.M (mast be between 0 and 1)
"""
def XYZ(m, Z=0.02):
    i = int(m*(len(hyd) - 1))
    if i > len(hyd) - 1:
        i = len(hyd) - 1
    X = hyd[i]
    Y = 1 - Z - X
    return (X, Y, Z)
    

In [52]:
def Opacity(den, T, X, Y, Z):
    T0 = 2e6
    return 0.4 + 2.3*(1 - math.exp(-pow(T/T0, 9/2)))*7e22*den*pow(T, -7/2)*(X + Y)*(X + 0.5*Y)


## Интегрирование от центра по расстоянию
параметры модели: плотность и температура в центре.

хим.состав, число точек, шаг по радиусу.

In [36]:
def model_R(d0, t0, Z=0.02, dim=30000, step=s.R/10000, M=s.M, debug=False) :
    dr = step  
    # distance from the center
    r = np.zeros(dim, dtype='float64')
    # density
    d = np.zeros(dim, dtype='float64')
    # temperature
    t = np.zeros(dim, dtype='float64')
    # светимость
    l = np.zeros(dim, dtype='float64')
    # mass inside r
    m = np.zeros(dim, dtype='float64')
    # pressure
    p = np.zeros(dim, dtype='float64')
    # hydrogen
    x = np.zeros(dim, dtype='float64')
    # opacity
    k = np.zeros(dim, dtype='float64')
    # 1 - convection, 0 - radiation
    c = np.zeros(dim)
    # скорость выгорания водорода (величина, обратная характерному времени)
    v = np.zeros(dim, dtype='float64')

# начальные данные
    for n in range(2) :
        r[n] = n*step
        V = 4*ph.pi/3*pow(r[n], 3)
        d[n] = d0
        t[n] = t0
        m[n] = V*d[n]
        X, Y, Z = XYZ(m[n]/M)
        x[n] = X
        mu = ph.MU(X, Y, Z)
        l[n] = m[n]*ph.Etot(d[n], t[n], X, Y, Z)
        p[n] = ph.Pressure(d[n], t[n], X, Y, Z)
        k[n] = Opacity(d[n], t[n], X, Y, Z)
        v[n] = 4*ph.m_prot*ph.Etot(d[n], t[n], X, Y, Z)/X/ph.dEpp*ph.year
        S = 4*ph.pi*r[n]*r[n]
        g = ph.G*m[n]/r[n]/r[n]
        rad = 3/16*k[n]*d[n]*l[n]/ph.sigma/pow(t[n], 3)/S
        con = 0.4*mu/ph.kB*ph.m_prot*g
        if rad < con:
            c[n] = 0.
        else:
            c[n] = 1.
    for n in range(1, dim - 1) :
        X, Y, Z = XYZ(m[n]/M)
        x[n] = X
        mu = ph.MU(X, Y, Z)
        S = 4*ph.pi*r[n]*r[n]
        g = ph.G*m[n]/r[n]/r[n]
        k[n] = Opacity(d[n], t[n], X, Y, Z)
        v[n] = 4*ph.m_prot*ph.Etot(d[n], t[n], X, Y, Z)/X/ph.dEpp*ph.year
        rad = -3/16*k[n]*d[n]*l[n]/ph.sigma/pow(t[n], 3)/S
        con = -0.4*mu/ph.kB*ph.m_prot*g
        r[n+1] = r[n] + dr
        x[n+1] = .732
        if abs(rad) < abs(con):
            c[n] = 0.
            dtdr = rad
        else:
            c[n] = 1.
            dtdr = con
        dt = 2*dtdr*dr - (t[n] - t[n-1])/(r[n] - r[n-1])*dr
        t[n+1] = t[n] + dt
        dpdr = -g*d[n]
#         p[n+1] = p[n] + 2*dpdr*dr - (p[n] -p[n-1])/(r[n] - r[n-1])*dr
        p[n+1] = p[n] + dpdr*dr
        # Находим плотность из уравнения состояния
        d[n+1] = mu*ph.m_prot*p[n+1]/ph.kB/t[n+1]
        davg = (d[n+1] + d[n])/2
        tavg = (t[n+1] + t[n])/2
        dm = 4*ph.pi/3*(pow(r[n+1], 3) - pow(r[n], 3))*davg
        m[n+1] = m[n] + dm
        l[n+1] = l[n] + dm*ph.Etot(davg, tavg, X, Y, Z)
        
        if debug:
            print("%d r=%.4f m=%.4f d=%.3f t=%.0f p=%.2e k=%.2f c=%.1f" % (n, r[n]/s.R, m[n]/M, d[n], t[n], p[n], k[n], c[n]))
        # уменьшаем шаг если температура падает слишком резко
        if abs(dt/t[n]) > 1e-3 :
            dr = 0.7*dr
        if m[n]/M > 1.2 : # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            raise Exception('Mass exceeded 1.2*M_target')
        Tsur = pow(l[n]/ph.sigma/S, 1/4)
        if t[n] < Tsur :
            break
        if p[n] < 0. : 
            raise Exception("Negative pressure")
            break

    if (n == dim - 2) :
         raise Exception("Error, increase dim!")

    df = pd.DataFrame({'Mass': m[:n], 'Radius': r[:n], 'Temperature': t[:n], 'Density': d[:n], 'Luminosity': l[:n], 
                       'Hydrogen': x[:n], 'Opacity': k[:n], 'Convection': c[:n], 'Pressure': p[:n], 'Velocity': v[:n]})

    return (n, m[n], r[n], l[n], t[n], df)

In [ ]:
%%time
t0=1.33300e+07 
d0=94.5544713300
n, M, R, L, T, df = model_R(d0, t0, debug=True)
n, M/s.M, R/s.R, L/s.L, T

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [54]:
%%time
dim = 11
for j in range(dim) :
#     T0 = 1.334e7 + 0.001e7*j  # M = M_sol, epoch = 0
    T0 = 14.50e6 + 0.01e6*j
    step = 10 # step by density
    exc = True
    den0 = 60
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
#             print('j=%d i=%d step=%.9f d=%.8f t=%.0f' % (j, i, step, den0, T0))
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
                den0 -= step
                step /= 10.
                break
            den0 -= step
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
    if M/s.M > 1 :
#         df = df_prev
        break
print("Done!")

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


	T0=1.45000e+07 D0=100.0000000000 n=10275 M=0.7835 R=0.4042 L=0.8813 T=8801
	T0=1.45000e+07 D0=91.0000000000 n=10476 M=0.8342 R=0.4285 L=0.8599 T=8495
	T0=1.45000e+07 D0=90.1000000000 n=10909 M=0.9263 R=0.4843 L=0.9148 T=8116
	T0=1.45000e+07 D0=90.0500000000 n=11436 M=0.9784 R=0.5580 L=0.9213 T=7572
	T0=1.45000e+07 D0=90.0480000000 n=11857 M=0.9954 R=0.6215 L=0.9216 T=7175
	T0=1.45000e+07 D0=90.0480000000 n=11857 M=0.9954 R=0.6215 L=0.9216 T=7175
	T0=1.45000e+07 D0=90.0479100000 n=12094 M=0.9992 R=0.6546 L=0.9217 T=6992
	T0=1.45000e+07 D0=90.0479080000 n=12107 M=0.9994 R=0.6590 L=0.9217 T=6971
	T0=1.45000e+07 D0=90.0479073000 n=12123 M=0.9995 R=0.6732 L=0.9217 T=6894
	T0=1.45000e+07 D0=90.0479072900 n=12119 M=0.9995 R=0.6756 L=0.9217 T=6882
# T0=1.45000e+07 D0=90.05 n=12119 M=0.9995 R=0.6756 L=0.922 T=6882
	T0=1.45100e+07 D0=100.0000000000 n=10276 M=0.7844 R=0.4043 L=0.8849 T=8808
	T0=1.45100e+07 D0=91.0000000000 n=10479 M=0.8359 R=0.4290 L=0.8644 T=8505
	T0=1.45100e+07 D0=90.100000000

In [55]:
# df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.451000e+07,90.073858,0.000000e+00,0.732,1.445415,1.0,1.794120e+17,1.208753e-10
1,1.272082e+23,6.960000e+06,1.451000e+07,90.073858,2.198077e+24,0.732,1.445415,0.0,1.794120e+17,1.208753e-10
2,1.017666e+24,1.392000e+07,1.450999e+07,90.073846,1.758459e+25,0.732,1.445416,0.0,1.794119e+17,1.208749e-10
3,3.434620e+24,2.088000e+07,1.450999e+07,90.073780,5.934786e+25,0.732,1.445417,0.0,1.794117e+17,1.208745e-10
4,8.141317e+24,2.784000e+07,1.450997e+07,90.073703,1.406759e+26,0.732,1.445420,0.0,1.794113e+17,1.208738e-10
...,...,...,...,...,...,...,...,...,...,...
12122,1.990588e+33,4.711120e+10,6.911885e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,2.000597e+07,3.977335e-48
12123,1.990588e+33,4.711123e+10,6.906126e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.996429e+07,3.937415e-48
12124,1.990588e+33,4.711127e+10,6.900365e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.992266e+07,3.897854e-48
12125,1.990588e+33,4.711130e+10,6.894606e+03,0.000021,3.562824e+33,0.732,0.400001,1.0,1.988109e+07,3.858666e-48


In [56]:
epoch = 0
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/3epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/3epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [ ]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)